## Contents

* [Setup](#Setup)
* [References](#References)

## Setup

In this notebook, we generate the experimental data required to carry out the statistical evaluation described in [1].
The data is logged into CSV files which will be later used when comparing the different MOCMA variants.

In [1]:
from itertools import product

import numpy as np
from IPython.display import Markdown

from anguilla.fitness import benchmark

from anguilla.evaluation import MOCMATrialParameters, LogParameters, log_mocma_trials

In [2]:
# Search space dimension for both constrained and non-rotated benchmark functions is 30.

D_CONSTRAINED_NONROTATED = 30

# Search space dimension for rotated benchmark functions is 10.

D_ROTATED = 10

N_PARENTS = 100

N_TRIALS = 25

# The cartesian product of the following three global variables represents the space of 
# configured MOCMA-Benchmark function pairs for which we need to run N_TRIALS each

FUNCTIONS = [
    (benchmark.ZDT1, [D_CONSTRAINED_NONROTATED], 0.6),
    (benchmark.ZDT2, [D_CONSTRAINED_NONROTATED], 0.6),
    (benchmark.ZDT3, [D_CONSTRAINED_NONROTATED], 0.6),
    (benchmark.ZDT4, [D_CONSTRAINED_NONROTATED], 0.6),
    (benchmark.ZDT6, [D_CONSTRAINED_NONROTATED], 0.6),
    (benchmark.IHR1, [D_ROTATED], 1.0),
    (benchmark.IHR2, [D_ROTATED], 1.0),
    (benchmark.IHR3, [D_ROTATED], 1.0),
    (benchmark.IHR4, [D_ROTATED], 1.0),
    (benchmark.IHR6, [D_ROTATED], 1.0),
    (benchmark.ELLI1, [D_ROTATED], 1.0),
    (benchmark.ELLI2, [D_ROTATED], 1.0),
    (benchmark.CIGTAB1, [D_ROTATED], 1.0),
    (benchmark.CIGTAB2, [D_ROTATED], 1.0),
    (benchmark.DTLZ1, [D_CONSTRAINED_NONROTATED, 3], 1.0),
    (benchmark.DTLZ2, [D_CONSTRAINED_NONROTATED, 3], 1.0),
    (benchmark.DTLZ3, [D_CONSTRAINED_NONROTATED, 3], 1.0),
    (benchmark.DTLZ4, [D_CONSTRAINED_NONROTATED, 3], 1.0),
    (benchmark.DTLZ5, [D_CONSTRAINED_NONROTATED, 3], 1.0),
    (benchmark.DTLZ6, [D_CONSTRAINED_NONROTATED, 3], 1.0),
    (benchmark.DTLZ7, [D_CONSTRAINED_NONROTATED, 3], 1.0),
    (benchmark.GELLI, [D_ROTATED, 3], 1.0),
]

# These next two global variables define the 4 MOCMA variants we will evaluate

OPTIMIZER_PARENT_OFFSPRING = [
    (100, 100),
    (100, 1),
]

OPTIMIZER_SUCCESS_NOTION = [
    "population",
    "individual",
]

# Generate the trail parameters
TRIAL_PARAMETERS = []
for configuration in product(FUNCTIONS, OPTIMIZER_PARENT_OFFSPRING, OPTIMIZER_SUCCESS_NOTION):
    (fn_cls, fn_args, initial_step_size), (n_parents, n_offspring), success_notion = configuration
    TRIAL_PARAMETERS.append(
        MOCMATrialParameters(
            fn_cls,
            fn_args=fn_args,
            n_parents=n_parents,
            success_notion=success_notion,
            initial_step_size=initial_step_size,
        )
    )

# For reproducible results we set a base seed to create the seed 
# sequence used to generate children seeds for each independent trial
SEED = 90508458

# We want to checkpoint the best solutions at 25K and 50K function evaluations
LOG_PARAMETERS = LogParameters("./output/logs", log_at=[25000, 50000])

In [3]:
Markdown("We need to run {} independent trials for each of the {} pairs, which totals {} jobs.".format(
    N_TRIALS,
    len(TRIAL_PARAMETERS),
    N_TRIALS * len(TRIAL_PARAMETERS)
))

We need to run 25 independent trials for each of the 88 pairs, which totals 2200 jobs.

In [4]:
log_mocma_trials(LOG_PARAMETERS,
           TRIAL_PARAMETERS,
           # If we partition with slices we can distribute the load between
           # multiple computers
           trial_slice=slice(0, 3),
           seed=SEED,
           n_trials=N_TRIALS,
          )

Number of jobs: 3

Number of processes: 3

Chunksize: 1

First job: MOCMATrialParameters(fn_cls=<class 'anguilla.fitness.benchmark.zdt.ZDT1'>, fn_args=[30], fn_kwargs={}, fn_rng_seed=None, n_parents=100, initial_step_size=0.6, success_notion='population', n_offspring=100, reference=None, region_bounds=None, max_generations=None, max_evaluations=None, target_indicator_value=None, key=1, seed=SeedSequence(
    entropy=90508458,
    spawn_key=(0,),
))

Last job: MOCMATrialParameters(fn_cls=<class 'anguilla.fitness.benchmark.zdt.ZDT1'>, fn_args=[30], fn_kwargs={}, fn_rng_seed=None, n_parents=100, initial_step_size=0.6, success_notion='population', n_offspring=100, reference=None, region_bounds=None, max_generations=None, max_evaluations=None, target_indicator_value=None, key=3, seed=SeedSequence(
    entropy=90508458,
    spawn_key=(2,),
))

Running 3 job(s) using parallel execution.
(100+100)-MO-CMA-ES-P-ZDT1-1
(100+100)-MO-CMA-ES-P-ZDT1-2
(100+100)-MO-CMA-ES-P-ZDT1-3


## References

[1] T. Voß, N. Hansen, and C. Igel. Improved Step Size Adaptation for the MO-CMA-ES. In Genetic And Evolutionary Computation Conference, 487–494. Portland, United States, July 2010. ACM. URL: https://hal.archives-ouvertes.fr/hal-00503251, doi:10.1145/1830483.1830573.